In [1]:
import numpy as np 
import pandas as pd 
import os, datetime 
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *  
from tensorflow.keras.callbacks import *
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold 
from tqdm import tqdm
import random 
import time
import pywt

In [2]:
## Load data 
train_features = pd.read_csv('drive/MyDrive/movement_detection/train_features.csv')
train_labels = pd.read_csv('drive/MyDrive/movement_detection/train_labels.csv')
test_features = pd.read_csv('drive/MyDrive/movement_detection/test_features.csv')
sample_submission = pd.read_csv('drive/MyDrive/movement_detection/sample_submission.csv')

In [3]:
X = tf.reshape(np.array(train_features.iloc[:,2:]),[-1, 600, 6])
X = np.asarray(X) 
X.shape

(3125, 600, 6)

In [4]:
y = train_labels['label'].values
y.shape

(3125,)

In [5]:
batch_size = 64
seq_len = 600 # temporary 
d_k = 256 
d_v = 256
n_heads = 12 
ff_dim = 512

In [6]:
class SingleAttention(Layer):
    def __init__(self, d_k, d_v):
        super(SingleAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v

    def build(self, input_shape):
        self.query = Dense(self.d_k, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')
    
        self.key = Dense(self.d_k, 
                     input_shape=input_shape, 
                     kernel_initializer='glorot_uniform', 
                     bias_initializer='glorot_uniform')
    
        self.value = Dense(self.d_v, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')

    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
        q = self.query(inputs[0])
        k = self.key(inputs[1])

        attn_weights = tf.matmul(q, k, transpose_b=True)
        attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
        attn_weights = tf.nn.softmax(attn_weights, axis=-1)
    
        v = self.value(inputs[2])
        attn_out = tf.matmul(attn_weights, v)
        return attn_out    

In [7]:
class MultiAttention(Layer):
    def __init__(self, d_k, d_v, n_heads):
        super(MultiAttention, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.attn_heads = list()

    def build(self, input_shape):
        for n in range(self.n_heads):
            self.attn_heads.append(SingleAttention(self.d_k, self.d_v))  
    
        # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1]=7 
        self.linear = Dense(input_shape[0][-1], 
                        input_shape=input_shape, 
                        kernel_initializer='glorot_uniform', 
                        bias_initializer='glorot_uniform')

    def call(self, inputs):
        attn = [self.attn_heads[i](inputs) for i in range(self.n_heads)]
        concat_attn = tf.concat(attn, axis=-1)
        multi_linear = self.linear(concat_attn)
        return multi_linear   


In [8]:
class TransformerEncoder(Layer):
    def __init__(self, d_k, d_v, n_heads, ff_dim, dropout=0.1, **kwargs):
        super(TransformerEncoder, self).__init__()
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.ff_dim = ff_dim
        self.attn_heads = []
        self.dropout_rate = dropout 

    def build(self, input_shape):
        self.attn_multi = MultiAttention(self.d_k, self.d_v, self.n_heads)
        self.attn_dropout = Dropout(self.dropout_rate)
        self.attn_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)

        self.ff_conv1D_1 = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')
        # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1] = 7 
        self.ff_conv1D_2 = Conv1D(filters=input_shape[0][-1], kernel_size=1) 
        self.ff_dropout = Dropout(self.dropout_rate)
        self.ff_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)    
  
    def call(self, inputs): # inputs = (in_seq, in_seq, in_seq) 
        attn_layer = self.attn_multi(inputs) 
        attn_layer = self.attn_dropout(attn_layer)
        attn_layer = self.attn_normalize(inputs[0] + attn_layer)
        ff_layer = self.ff_conv1D_1(attn_layer)  
        ff_layer = self.ff_conv1D_2(ff_layer)
        ff_layer = self.ff_dropout(ff_layer)
        ff_layer = self.ff_normalize(inputs[0] + ff_layer)
        return ff_layer 

    def get_config(self): # Needed for saving and loading model with custom layer
        config = super().get_config().copy()
        config.update({'d_k': self.d_k,
                   'd_v': self.d_v,
                   'n_heads': self.n_heads,
                   'ff_dim': self.ff_dim,
                   'attn_heads': self.attn_heads,
                   'dropout_rate': self.dropout_rate})
        return config


In [9]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [10]:
def build_model(seq_len, features): 
  inputs = Input(shape=(seq_len,features))   
  bn = BatchNormalization()(inputs)    

  conv = Conv1D(128, 5, activation = 'relu', padding = 'same')(bn) 
  bn = BatchNormalization()(conv) 
  maxpool = MaxPooling1D()(bn) 
  avgpool = AveragePooling1D()(bn) 
  pooling = Concatenate()([maxpool,avgpool])   
  conv = Conv1D(128, 5, activation = 'relu', padding = 'same')(pooling) 
  bn = BatchNormalization()(conv) 
  maxpool = MaxPooling1D()(bn) 
  avgpool = AveragePooling1D()(bn) 
  pooling = Concatenate()([maxpool, avgpool]) 


  pos_encoding = positional_encoding(seq_len, d_k)   
  pooling *= tf.math.sqrt(tf.cast(d_k, tf.float32)) 
  pooling += pos_encoding[:, :pooling.shape[1], :]

  attn_layer1 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer2 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer3 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)

  x = attn_layer1((bn, bn, bn))  
  x = attn_layer2((x, x, x)) 
  x = attn_layer3((x, x, x)) 

  bi_gru = Bidirectional(GRU(128, return_sequences = False))(x) 
  dropout = Dropout(0.25)(bi_gru)
  dense = Dense(128, activation = 'relu')(dropout) 
  bn = BatchNormalization()(dense) 
  outputs = Dense(61, activation='softmax')(bn) 
  model = Model(inputs=inputs, outputs=outputs)
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [11]:
model = build_model(600,6)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 600, 6)]     0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 600, 6)       24          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 600, 128)     3968        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 600, 128)     512         conv1d[0][0]                     
______________________________________________________________________________________________

In [12]:
kfold = StratifiedKFold(n_splits = 10, random_state = 960418, shuffle = True)
for idx,(train_idx, val_idx) in enumerate(kfold.split(X,y)):   
    print("... Validating on fold {} ...".format(idx+1))  
    if idx == 0: 
      continue # save time 
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx] 
    
    ##### augment data #####
    print("... Augmenting Data ...")
    X_augmented = [] 
    y_augmented = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True): 
        for j in range(10): 
            shifted = np.roll(X_train[i], int(random.random() * 600)) 
            X_augmented.append(shifted) 
            y_augmented.append(y_train[i]) 
        for j in range(10): 
            noised = np.random.normal(0, 1, X_train[i].shape) + X_train[i] 
            X_augmented.append(noised) 
            y_augmented.append(y_train[i]) 
    X_augmented = np.asarray(X_augmented) 
    y_augmented = np.asarray(y_augmented)
    X_train = np.concatenate([X_train, X_augmented]) 
    y_train = np.concatenate([y_train, y_augmented])   


    ###### feature engineering data ##### 
    print("... DFFT Feature Engineering ...")
    X_fourier_real = [] 
    X_fourier_imag = [] 
    for i in tqdm(range(X_train.shape[0]), position = 0, leave = True):  
        real_part = np.fft.fft(X_train[i]).real 
        imag_part = np.fft.fft(X_train[i]).imag 
        X_fourier_real.append(real_part)
        X_fourier_imag.append(imag_part) 
    
    X_fourier_real = np.asarray(X_fourier_real)  
    X_fourier_imag = np.asarray(X_fourier_imag)
    
    
    X_val_fourier_real = [] 
    X_val_fourier_imag = [] 
    for i in tqdm(range(X_val.shape[0]), position = 0, leave = True):
        real_part = np.fft.fft(X_val[i]).real 
        imag_part = np.fft.fft(X_val[i]).imag 
        X_val_fourier_real.append(real_part) 
        X_val_fourier_imag.append(imag_part)
    
    X_val_fourier_real = np.asarray(X_val_fourier_real) 
    X_val_fourier_imag = np.asarray(X_val_fourier_imag)
    
    X_train = np.concatenate([X_train, X_fourier_real, X_fourier_imag], axis = 2)  
    X_val = np.concatenate([X_val, X_val_fourier_real, X_val_fourier_imag], axis = 2)


    ##### train model #####  
    print("... Building Model ...")
    # we have 18 features after feature engineering 
    model = build_model(600, 18) 
    print("... Training ...") 
    model_path = 'drive/MyDrive/movement_detection/kfold' + str(idx+1) + '/testNoise_epoch_{epoch:03d}_val_{val_loss:.3f}_accuracy_{val_accuracy:.3f}.h5'
    learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', patience = 2, verbose = 1, factor = 0.8)
    checkpoint = ModelCheckpoint(filepath = model_path, monitor = 'val_loss', verbose = 1, save_best_only = True)
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 5) 

    model.fit(X_train,
              y_train,
              epochs = 200,
              batch_size = 32, # to avoid OOM when using colab 
              validation_data = (X_val, y_val), 
              callbacks = [learning_rate_reduction, checkpoint, early_stopping])



  0%|          | 0/2812 [00:00<?, ?it/s]

... Validating on fold 1 ...
... Validating on fold 2 ...
... Augmenting Data ...


  2%|▏         | 1034/59052 [00:00<00:05, 10334.93it/s]

... DFFT Feature Engineering ...


100%|██████████| 313/313 [00:00<00:00, 10419.51it/s]


...


Epoch 00007: val_loss did not improve from 1.10715
... Validating on fold 10 ...
... Augmenting Data ...


  2%|▏         | 1025/59073 [00:00<00:05, 10248.84it/s]

... DFFT Feature Engineering ...


100%|██████████| 312/312 [00:00<00:00, 10725.71it/s]


... Building Model ...
... Training ...
Epoch 1/200
1847/1847 [==============================] - 560s 291ms/step - loss: 2.2294 - accuracy: 0.5081 - val_loss: 1.3343 - val_accuracy: 0.6603

Epoch 00001: val_loss improved from inf to 1.33433, saving model to drive/MyDrive/movement_detection/kfold10/testNoise_epoch_001_val_1.334_accuracy_0.660.h5
Epoch 2/200
1847/1847 [==============================] - 532s 288ms/step - loss: 1.2431 - accuracy: 0.6429 - val_loss: 1.0093 - val_accuracy: 0.7179

Epoch 00002: val_loss improved from 1.33433 to 1.00927, saving model to drive/MyDrive/movement_detection/kfold10/testNoise_epoch_002_val_1.009_accuracy_0.718.h5
Epoch 3/200
1847/1847 [==============================] - 535s 290ms/step - loss: 0.7835 - accuracy: 0.7646 - val_loss: 0.9961 - val_accuracy: 0.7436

Epoch 00003: val_loss improved from 1.00927 to 0.99606, saving model to drive/MyDrive/movement_detection/kfold10/testNoise_epoch_003_val_0.996_accuracy_0.744.h5
Epoch 4/200
1847/1847 [========

## Make Prediction

In [14]:
model1 = load_model('drive/MyDrive/movement_detection/kfold1/testNoise_epoch_004_val_0.905_accuracy_0.757.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model2 = load_model('drive/MyDrive/movement_detection/kfold2/testNoise_epoch_002_val_1.010_accuracy_0.725.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model3 = load_model('drive/MyDrive/movement_detection/kfold3/testNoise_epoch_004_val_0.957_accuracy_0.751.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})  
model4 = load_model('drive/MyDrive/movement_detection/kfold4/testNoise_epoch_002_val_0.994_accuracy_0.709.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model5 = load_model('drive/MyDrive/movement_detection/kfold5/testNoise_epoch_004_val_0.966_accuracy_0.748.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model6 = load_model('drive/MyDrive/movement_detection/kfold6/testNoise_epoch_004_val_0.854_accuracy_0.772.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model7 = load_model('drive/MyDrive/movement_detection/kfold7/testNoise_epoch_003_val_1.004_accuracy_0.731.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model8 = load_model('drive/MyDrive/movement_detection/kfold8/testNoise_epoch_004_val_0.859_accuracy_0.763.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})
model9 = load_model('drive/MyDrive/movement_detection/kfold9/testNoise_epoch_002_val_1.107_accuracy_0.734.h5',
                    custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder}) 
model10 = load_model('drive/MyDrive/movement_detection/kfold10/testNoise_epoch_004_val_0.966_accuracy_0.763.h5',
                     custom_objects={'SingleAttention': SingleAttention,
                                    'MultiAttention': MultiAttention,
                                    'TransformerEncoder': TransformerEncoder})


In [16]:
X_test = tf.reshape(np.array(test_features.iloc[:,2:]),[-1, 600, 6])
X_test = np.asarray(X_test)

##### feature engineering for test dataset ##### 
X_test_fourier_real = [] 
X_test_fourier_imag = [] 
for i in tqdm(range(X_test.shape[0]), position = 0, leave = True):
    real_part = np.fft.fft(X_test[i]).real 
    imag_part = np.fft.fft(X_test[i]).imag 
    X_test_fourier_real.append(real_part) 
    X_test_fourier_imag.append(imag_part)
    
X_test_fourier_real = np.asarray(X_test_fourier_real) 
X_test_fourier_imag = np.asarray(X_test_fourier_imag)
X_test = np.concatenate([X_test, X_test_fourier_real, X_test_fourier_imag], axis = 2)  

print(X_test.shape)


100%|██████████| 782/782 [00:00<00:00, 9589.11it/s]

(782, 600, 18)


In [17]:
X_test.shape

(782, 600, 18)

In [18]:
p1 = model1.predict(X_test) 
p2 = model2.predict(X_test)
p3 = model3.predict(X_test) 
p4 = model4.predict(X_test) 
p5 = model5.predict(X_test) 
p6 = model6.predict(X_test) 
p7 = model7.predict(X_test) 
p8 = model8.predict(X_test)
p9 = model9.predict(X_test)
p10 = model10.predict(X_test)

In [21]:
p_avg = (p1 + p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9 + p10)/10.0

sample_submission.iloc[:,1:] = p_avg

sample_submission.to_csv("drive/MyDrive/movement_detection/fourier_transform_10_fold_transformer.csv",index=False)


In [22]:
sample_submission.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.000047,0.006766,1.274405e-05,0.000118,0.000082,0.000011,0.327663,0.001213,0.000051,0.140336,0.022922,0.029442,0.000855,0.160345,5.897850e-03,0.000356,0.000219,0.000214,0.000045,0.000016,7.206438e-06,1.680275e-04,0.000221,0.000109,0.024648,0.000002,0.000256,6.882704e-04,0.000053,0.000261,0.028017,5.909244e-03,0.000007,0.000473,0.003883,0.000002,0.000966,0.076400,0.000166,0.000057,0.000490,0.000591,0.053101,0.032264,0.004587,0.025970,0.000018,2.043550e-02,0.000029,0.000011,0.000004,5.291438e-04,0.000618,7.303744e-05,2.842282e-03,7.567881e-06,0.000057,0.000274,0.019107,0.000016,0.000069
1,3126,0.000287,0.000031,3.575826e-06,0.000050,0.000015,0.000071,0.000004,0.000088,0.000005,0.000003,0.000011,0.000019,0.000003,0.000008,5.010917e-06,0.000191,0.000013,0.000004,0.000034,0.000002,7.410972e-06,1.197676e-04,0.001090,0.001671,0.000052,0.000009,0.992444,3.680520e-05,0.000005,0.000006,0.000003,6.448591e-07,0.000089,0.000037,0.000022,0.000678,0.001523,0.000037,0.000005,0.000004,0.000185,0.000015,0.000010,0.000003,0.000005,0.000013,0.000002,4.741729e-05,0.000202,0.000315,0.000059,2.698824e-06,0.000001,1.006968e-06,4.294962e-06,1.179097e-05,0.000030,0.000318,0.000003,0.000005,0.000080
2,3127,0.000098,0.000379,1.956535e-05,0.000081,0.000450,0.000015,0.025156,0.000222,0.003722,0.075049,0.056015,0.003405,0.003389,0.001356,4.279258e-01,0.001015,0.000035,0.000365,0.000013,0.000055,3.291884e-05,1.049551e-04,0.000026,0.000009,0.005655,0.000165,0.004865,8.511020e-05,0.001604,0.004882,0.003581,9.442770e-03,0.007016,0.003681,0.018096,0.000024,0.000929,0.001656,0.013037,0.003420,0.000035,0.000060,0.049454,0.104345,0.000613,0.078746,0.000025,5.417238e-03,0.016926,0.000522,0.008929,1.289418e-03,0.004755,7.168115e-05,5.362349e-04,1.005875e-05,0.000016,0.000022,0.004715,0.000300,0.046137
3,3128,0.000058,0.000009,1.191381e-04,0.000071,0.000002,0.000189,0.000013,0.001359,0.000097,0.000007,0.000093,0.000045,0.000009,0.000006,1.056316e-05,0.000077,0.000001,0.000001,0.000011,0.000015,2.905080e-05,1.826855e-05,0.000226,0.001106,0.000099,0.000043,0.953906,1.877368e-05,0.000092,0.000184,0.000002,5.384193e-07,0.013274,0.000068,0.000009,0.000105,0.000045,0.000023,0.000152,0.000020,0.000087,0.000030,0.000060,0.000249,0.000140,0.000055,0.000014,2.842437e-05,0.004239,0.018863,0.000035,2.435880e-05,0.000008,5.359575e-06,1.680417e-05,2.810935e-05,0.000022,0.000210,0.000043,0.000025,0.004202
4,3129,0.012303,0.000100,2.353196e-07,0.000452,0.000269,0.000052,0.000005,0.000007,0.000013,0.000001,0.000005,0.000023,0.000002,0.000006,5.345769e-07,0.000040,0.000004,0.000066,0.000227,0.000005,7.012329e-07,4.301604e-07,0.000187,0.000029,0.000171,0.000278,0.982874,2.383607e-07,0.000003,0.000007,0.000001,8.383294e-07,0.000033,0.000034,0.000032,0.000003,0.000007,0.000004,0.000031,0.000004,0.000004,0.000860,0.000138,0.000039,0.000004,0.000029,0.000013,4.359752e-07,0.000052,0.000314,0.000130,9.598443e-08,0.000002,6.931358e-07,6.351819e-07,7.954644e-07,0.000044,0.000098,0.000009,0.000917,0.000063
